In [20]:
from collections import Counter
from flask import Flask, request, render_template, jsonify
import pickle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import argparse
import time
from splinter import Browser
import time
from collections import defaultdict
import json
import re

In [10]:
data = pd.read_csv("data/bb_reference/2016to2019Data_merged.csv", encoding = 'latin-1')

In [4]:
salary_data = pd.read_csv('data/salary_data_dk.csv')

In [11]:
data = data.drop(['Unnamed: 0_y','Unnamed: 0'], axis = 1)
#data.columns[1]= 'gamescore_rank'

TypeError: Index does not support mutable operations

In [13]:
data = data.rename(columns={'Unnamed: 0_x':'gamescore_rank'})

In [14]:
salary_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75048 entries, 0 to 75047
Data columns (total 4 columns):
player    75048 non-null object
pos       75048 non-null object
salary    75048 non-null int64
date      75048 non-null object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [15]:
from src.cleaner import Cleaner 
import src.fantasy_scoring as fs

In [16]:
clnr = Cleaner()
data = clnr.bb_ref_to_dk(data)
data['dk_score']= data.apply(lambda x: fs.dk_score(x),axis = 1)

In [17]:
data['dk_per_min']=data.apply(lambda x: 0 if x['seconds_played']==0 else x['dk_score']*60/x['seconds_played'],axis = 1)
data['minutes']=data['seconds_played']/60
data['fg_pct']=data.apply(lambda x: 0 if x['attempted_field_goals']==0 
                          else x['made_field_goals']/x['attempted_field_goals'],
                          axis = 1)
data['three_pct']=data.apply(lambda x: 0 if x['attempted_three_point_field_goals']==0 
                          else (x['made_three_point_field_goals'])/x['attempted_three_point_field_goals'],
                          axis = 1)
data['two_pct']=data.apply(lambda x: 0 if (x['attempted_field_goals']-x['attempted_three_point_field_goals'])==0 
                          else (x['made_field_goals']-x['made_three_point_field_goals'])/
                           (x['attempted_field_goals']-x['attempted_three_point_field_goals']),
                          axis = 1)
data['ft_pct']=data.apply(lambda x: 0 if x['attempted_free_throws']==0 
                          else x['made_free_throws']/x['attempted_free_throws'],
                          axis = 1)

In [18]:
data['w']=data['outcome'].apply(lambda x: 1 if x == 'Outcome.WIN' else 0)
data['dk_per_min']=data.apply(lambda x: 0 if x['seconds_played']==0 else x['dk_score']*60/x['seconds_played'],axis = 1)

In [21]:
def get_n_scatter(data, f1, f2,n=1500,buckets = 10,ax = plt, ascending = True,alpha=0.7, s = 6):
    sub = data.iloc[::int(len(data)/n),:]
    lines= np.r_[0:len(sub):int(len(sub)/buckets)]
    out = np.zeros((buckets,2))
    temp_data = sub.sort_values(by = f1,ascending=ascending)
    #print(temp_data.shape)
    for ind,i in enumerate(lines[1:]):
        quantile = temp_data[f1].iloc[i]
        out[ind,0]=quantile
        sub = temp_data[(temp_data[f1]>=temp_data[f1].iloc[lines[ind]])&
                             (temp_data[f1]<=quantile)]
        #print(sub.shape)
        out[ind,1]=sub[f2].mean()
        ax.scatter(sub[f1],sub[f2],alpha = alpha, s = s )
    ax.set_xlabel(f1, size = 16)
    ax.set_ylabel(f2, size = 16)
    return pd.DataFrame(out,columns=[f1,f2])

In [24]:
data['dt']=pd.to_datetime(data['date'])
salary_data['dt']=pd.to_datetime(salary_data['date'])

In [64]:
salary_data['dt']=pd.to_datetime(salary_data['date'])

In [25]:
min_date = salary_data['dt'].min()
both_data = data[data['dt']>=min_date]
both_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55650 entries, 138 to 101013
Data columns (total 44 columns):
gamescore_rank                       55650 non-null int64
assists                              55650 non-null int64
attempted_field_goals                55650 non-null int64
attempted_free_throws                55650 non-null int64
attempted_three_point_field_goals    55650 non-null int64
blocks                               55650 non-null int64
date                                 55650 non-null object
defensive_rebounds                   55650 non-null int64
game_score                           55650 non-null float64
location                             55650 non-null object
made_field_goals                     55650 non-null int64
made_free_throws                     55650 non-null int64
made_three_point_field_goals         55650 non-null int64
name                                 55650 non-null object
offensive_rebounds                   55650 non-null int64
opponent     

In [23]:
grp = data[data['minutes']>5].groupby(['name'])
summary=grp.aggregate({
    'dt':len,
    'minutes':['sum',np.mean],
    'td':['sum',np.mean],
    'dd':['sum',np.mean],
    'doubles':['sum',np.mean],
    'dk_per_min':[np.mean,np.median,'max',np.std],
    'dk_per_k':[np.mean,np.median,'max',np.std],
    'dk_score':[np.mean,np.median,'max'],
    'salary': [np.mean,np.median,'max',np.std],
    'pos':pd.Series.mode,
    'points':[np.mean,'max',np.std],
    'game_score':[np.mean,'max',np.std],
    'fg_pct':np.mean,
    'three_pct':np.mean,
    'two_pct':np.mean,
    'rebounds':np.mean,
    'offensive_rebounds':np.mean,
    'defensive_rebounds':np.mean,
    'assists':np.mean,
    'steals':np.mean,
    'blocks':np.mean,
    'turnovers':np.mean,
    'w':np.mean
}).sort_values(by=('dk_per_min','mean'),ascending=False)

KeyError: "Column 'dt' does not exist!"

In [46]:
f = open('dk_contest_dates2017-2019.csv','r')
date_dict = defaultdict(str)
for line in f.readlines():
    date = line.split(',')[0]
    slash_date = "/".join([date.split('-')[1],date.split('-')[2],date.split('-')[0]])
    date_dict[line.split(',')[0]]=line.split(',')[1].strip()
    date_dict[slash_date]=line.split(',')[1].strip()

In [47]:
date_dict['2017-10-18']

'2017-10-18'

In [42]:
both_data['date']=both_data['dt'].apply(lambda x: x.strftime("%Y-%m-%d"))

/Users/shane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [50]:
both_data['contest_date']=both_data['date'].apply(lambda x: date_dict[x])

In [61]:
both_data = both_data[(both_data['contest_date'].apply(lambda x: len(x))!=0)]


In [63]:
gaps = pd.read_csv('data/salary_data_gaps.csv')
gaps.info()
salary_data = salary_data.append(gaps)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9250 entries, 0 to 9249
Data columns (total 4 columns):
player    9250 non-null object
pos       9250 non-null object
salary    9250 non-null int64
date      9250 non-null object
dtypes: int64(1), object(3)
memory usage: 289.1+ KB


/Users/shane/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [65]:
print(salary_data.shape)
print(both_data.shape)

(84298, 5)
(55394, 45)


In [66]:
left = both_data.merge(salary_data,how = 'left',left_on=['name','dt'],right_on=['player','dt'])
right = both_data.merge(salary_data,how = 'right',left_on=['name','dt'],right_on=['player','dt'])

In [85]:
left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55394 entries, 0 to 55393
Data columns (total 49 columns):
gamescore_rank                       55394 non-null int64
assists                              55394 non-null int64
attempted_field_goals                55394 non-null int64
attempted_free_throws                55394 non-null int64
attempted_three_point_field_goals    55394 non-null int64
blocks                               55394 non-null int64
date_x                               55394 non-null object
defensive_rebounds                   55394 non-null int64
game_score                           55394 non-null float64
location                             55394 non-null object
made_field_goals                     55394 non-null int64
made_free_throws                     55394 non-null int64
made_three_point_field_goals         55394 non-null int64
name                                 55394 non-null object
offensive_rebounds                   55394 non-null int64
opponent        

In [78]:
#left['salary'].value_counts()
#left.info()
np.min(right['name'].apply(lambda x: len(x)))

TypeError: object of type 'float' has no len()

In [101]:
empties = left[['name','player_y','slug','date_x']].drop_duplicates('slug')
empties.to_csv("data/dk_player_map.csv")

In [103]:
empties_dk = right[right['name'].isna()][['name','player_y','slug','date_y']]


In [106]:
empties_dk.drop_duplicates('player_y').to_csv('data/dk_empties.csv')